In [3]:
import numpy as np
import pandas as pd
%matplotlib notebook

In [4]:
n = str(2)
fullDF = pd.read_csv("data/Chiller"+n+"_full.csv")
num_samples = fullDF.shape[0]
window_size = 60

In [5]:
fullDF.head()

,ts,ch1Watt,ch2Watt,ch3Watt,value1,value2,value3,value4,conflowRate,conflowSpeed,evaflowRate,evaflowSpeed
0,2017-05-01 00:00:00,325,0,324,18.580934,21.582337,23.633118,23.451539,0.0,0.0,0.0,0.0
1,2017-05-01 00:01:00,326,0,328,18.602295,21.539612,23.558351,23.398133,0.0,0.0,0.0,0.0
2,2017-05-01 00:02:00,327,0,324,18.495484,21.614379,23.483581,23.323363,0.0,0.0,0.0,0.0
3,2017-05-01 00:03:00,338,0,325,18.474123,21.571657,23.601076,23.205871,0.0,0.0,0.0,0.0
4,2017-05-01 00:04:00,334,0,328,18.559570,21.560973,23.536987,23.227236,0.0,0.0,0.0,0.0


In [6]:
Watt = fullDF[["ch1Watt", "ch2Watt", "ch3Watt"]].as_matrix()
Value = fullDF[["value1", "value2", "value3", "value4"]].as_matrix()
Conflow = fullDF[["conflowRate", "conflowSpeed"]].as_matrix()
Evaflow = fullDF[["evaflowRate", "evaflowSpeed"]].as_matrix()
Total_Watt = np.sum(Watt, axis = 1)

In [7]:
from sklearn import preprocessing

In [8]:
Watt_scaled = preprocessing.scale(Watt)
Value_scaled = preprocessing.scale(Value)
Conflow_scaled = preprocessing.scale(Conflow)
Evaflow_scaled = preprocessing.scale(Evaflow)
Total_Watt_scaled = preprocessing.scale(Total_Watt)

/Users/duynguyen/anaconda/envs/python2/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [9]:
Hour = pd.to_datetime(fullDF.ts).dt.hour.as_matrix()
Hour_transformed = np.zeros([num_samples,24])
for i in range(num_samples):
    Hour_transformed[i, Hour[i]] = 1

In [10]:
Day = pd.to_datetime(fullDF.ts).dt.day.as_matrix()
Day_transformed = np.zeros([num_samples,31])
for i in range(num_samples):
    Day_transformed[i, Day[i] - 1] = 1

In [11]:
Month = pd.to_datetime(fullDF.ts).dt.month.as_matrix()
Month_transformed = np.zeros([num_samples,12])
for i in range(num_samples):
    Month_transformed[i, Month[i] - 1] = 1

In [65]:
# Data = np.concatenate([Hour_transformed, Value_scaled, Conflow_scaled, Evaflow_scaled], axis =1)
Data = np.concatenate([Value_scaled, Conflow_scaled, Evaflow_scaled], axis =1)

In [66]:
dimension = Data.shape[1]

In [67]:
Data_align = np.zeros((window_size, num_samples - window_size + 1, dimension))
for i in range(0, window_size):
    Data_align[i] = Data[i: num_samples - window_size + 1 + i]

In [68]:
Full_Data = Data_align.transpose((1,0,2))

In [69]:
Power = Total_Watt[:num_samples - window_size + 1].reshape((-1,1))

In [70]:
import keras
from keras import layers
from keras.layers import Lambda, Activation,recurrent, Bidirectional, Dense, Flatten, Conv1D, Dropout, LSTM, GRU, concatenate, multiply, add, Reshape, MaxPooling1D, BatchNormalization
from keras.models import Model, load_model
import keras.backend as K

In [71]:
def mean_pred(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true)/y_true)

In [72]:
inp = layers.Input(shape=(window_size, dimension), dtype=Full_Data.dtype)

In [73]:
enc1 = LSTM(50,return_sequences=True)(inp)
enc2 = LSTM(50,return_sequences=True)(enc1)
flatten = Flatten()(enc2)
dense1 = Dense(500, activation='relu')(flatten)

In [74]:
output = Dense(1)(dense1)

In [75]:
model = Model(inp,output)
model.compile(optimizer='adam',
              loss='mean_absolute_error',
              metrics=[mean_pred])

In [76]:
test_inp = Full_Data[:40000]
test_out = Power[:40000]
val_inp = Full_Data[40000:45000]
val_out = Power[40000:45000]

In [77]:
model.fit([test_inp], [test_out],
          epochs=5,
          validation_data=(val_inp, val_out))

Train on 40000 samples, validate on 5000 samples
Epoch 1/5
40000/40000 [==============================] - 409s - loss: 14272.4418 - mean_pred: 0.2360 - val_loss: 3607.7179 - val_mean_pred: 0.2153
Epoch 2/5
40000/40000 [==============================] - 396s - loss: 2390.1323 - mean_pred: 0.0850 - val_loss: 1588.9157 - val_mean_pred: 0.1315
Epoch 3/5
40000/40000 [==============================] - 395s - loss: 2148.1623 - mean_pred: 0.0638 - val_loss: 1396.9066 - val_mean_pred: 0.1152
Epoch 4/5
40000/40000 [==============================] - 403s - loss: 2098.3412 - mean_pred: 0.0564 - val_loss: 1548.3396 - val_mean_pred: 0.0608
Epoch 5/5
40000/40000 [==============================] - 407s - loss: 2032.0767 - mean_pred: 0.0370 - val_loss: 1311.6876 - val_mean_pred: 0.0428


In [78]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 60, 8)             0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 60, 50)            11800     
_________________________________________________________________
lstm_12 (LSTM)               (None, 60, 50)            20200     
_________________________________________________________________
flatten_5 (Flatten)          (None, 3000)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 500)               1500500   
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 501       
Total params: 1,533,001
Trainable params: 1,533,001
Non-trainable params: 0
_________________________________________________________________


In [79]:
pred = model.predict(val_inp, verbose=1)

5000/5000 [==============================] - 10s    


In [80]:
pred

array([[ 101593.71875  ],
       [ 103745.1796875],
       [ 102540.2890625],
       ..., 
       [ 155360.453125 ],
       [ 154467.515625 ],
       [ 153469.109375 ]], dtype=float32)

In [81]:
a = (pred - val_out)/val_out

In [89]:
b = a > 1

In [90]:
np.sum(b)

0

In [62]:
(pred[4878]-val_out[4878])/val_out[4878]

array([ 163.46547917])

In [63]:
val_out[4878]

array([750])

In [64]:
test_inp.shape

(40000, 60, 32)